In [37]:
import pandas as pd
import numpy as np
etf_codelist = pd.read_csv("tw_maincode_etf__2025.csv")
list_ = etf_codelist['code'].values
_price = pd.read_parquet("tw_price.parquet")
op_df = _price.pivot_table(index='da', columns="code", values='op')
hi_df = _price.pivot_table(index='da', columns="code", values='hi')
lo_df = _price.pivot_table(index='da', columns="code", values='lo')
cl_df = _price.pivot_table(index='da', columns="code", values='cl')
vol_df = _price.pivot_table(index='da', columns="code", values='vol')
to_df = _price.pivot_table(index='da', columns="code", values='turnover')

In [44]:
start_da = '2020-01-01'
end_da = "2020-04-01"
to_df = to_df[to_df.columns.intersection(list_)].loc[start_da:end_da, :]
op_df = op_df[op_df.columns.intersection(list_)].loc[start_da:end_da, :]
cl_df = cl_df[cl_df.columns.intersection(list_)].loc[start_da:end_da, :]
hi_df = hi_df[hi_df.columns.intersection(list_)].loc[start_da:end_da, :]
lo_df = lo_df[lo_df.columns.intersection(list_)].loc[start_da:end_da, :]

In [51]:
hi_lo_log = np.log(hi_df / lo_df)
next_day_log = np.log(cl_df / op_df).shift(-1)

In [55]:
std = hi_lo_log.std()
profit = hi_lo_log.sum(axis=0)
len(std), len(std.dropna()), len(profit), len(profit[profit != 0.0])

(280, 187, 280, 186)

In [84]:
# R反L槓桿, K, U指數股票型, B+C債
list_exclude = []
list_include = []
std_test = std.dropna()
profit_test = profit[profit != 0]

# test for the first time
for i in std_test.index:
    if i not in profit_test.index: print(i); list_exclude.append(i); continue;
    if "L" in i: list_exclude.append(i)
    elif "B" in i: list_exclude.append(i)
    elif "C" in i: list_exclude.append(i)
    elif "R" in i: list_exclude.append(i)
    else: list_include.append(i)

std_test = std_test[std_test.index.intersection(list_include)]
profit_test = profit_test[profit_test.index.intersection(list_include)]

# test again
for i in std_test.index:
    if i not in profit_test.index: print(i);

len(std_test), len(profit_test)
arr_test = np.column_stack((std_test, profit_test))

00643K TT Equity


In [15]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import MinCovDet
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 下载股票数据
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'AMD', 'BA', 'NFLX']
start_date = '2020-01-01'
end_date = '2025-01-01'
data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']

# 计算每日收益率和波动度（标准差）
returns = data.pct_change().dropna()  # 计算每日收益率
volatility = returns.std()  # 计算每日收益率的标准差作为波动度

# 计算收益的均值
mean_returns = returns.mean()

# 将波动度和收益合并成特征
features = pd.DataFrame({
    'Return': mean_returns,
    'Volatility': volatility
})

# 标准化特征
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 计算Mahalanobis距离
cov = np.cov(features_scaled.T)
inv_cov = np.linalg.inv(cov)

# 使用MinCovDet方法来得到Mahalanobis距离的估计
mcd = MinCovDet().fit(features_scaled)
mahalanobis_distances = mcd.mahalanobis(features_scaled)

# 使用KNN进行分群
knn = KNeighborsClassifier(n_neighbors=3, metric='mahalanobis', metric_params={'V': inv_cov})
knn.fit(features_scaled, mahalanobis_distances)

# 分群预测
clusters = knn.predict(features_scaled)

# 可视化结果
plt.scatter(features_scaled[:, 0], features_scaled[:, 1], c=clusters, cmap='viridis')
plt.xlabel('Return')
plt.ylabel('Volatility')
plt.title('Stock Clusters using KNN + Mahalanobis Distance')
plt.show()


[*********************100%%**********************]  10 of 10 completed


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [1]:
import pandas as pd
df = pd.read_parquet("2025_v1_tickers_test.parquet")

In [4]:
df.head(1)

,date,open,high,low,close,volume,symbol
0,2023-12-29,25.02,25.04,24.96,24.96,59,AAA


In [7]:
op_df = df.pivot_table(index='date', columns="symbol", values='open')
hi_df = df.pivot_table(index='date', columns="symbol", values='high')
lo_df = df.pivot_table(index='date', columns="symbol", values='low')
cl_df = df.pivot_table(index='date', columns="symbol", values='close')
vol_df = df.pivot_table(index='date', columns="symbol", values='volume')